# Projet Article 1

In [1]:
import pandas as pd
import numpy as np

## Creating the training Dataset :

In [2]:
df_to_moderate=pd.read_csv("selected_tweets.csv")
df_to_moderate.head()

,Id,tweet (without @),source,Harmful,Looked up word
0,1,sale arabe ( arrête avec tes hontes toi ),1.0,1.0,arabe
1,2,y'a une meuf dans classe elle se pleins du rac...,1.0,1.0,arabes
2,3,jparle de qui jveux quand elles se font baiser...,1.0,1.0,baiser
3,4,tellement sexy franchement t'ai baisable ^^,0.0,1.0,baisable
4,5,Je te prend matin midi et soir biatch comme si...,0.0,1.0,biatch


In [3]:
df_to_moderate=df_to_moderate.drop(["Id","source","Looked up word"],axis=1)
df_to_moderate.head()

,tweet (without @),Harmful
0,sale arabe ( arrête avec tes hontes toi ),1.0
1,y'a une meuf dans classe elle se pleins du rac...,1.0
2,jparle de qui jveux quand elles se font baiser...,1.0
3,tellement sexy franchement t'ai baisable ^^,1.0
4,Je te prend matin midi et soir biatch comme si...,1.0


In [4]:
df_article1_messages=pd.read_excel("AmauryModerationAllMessagesInspireFrom3Aout2020.xlsx", sheet_name="ContenuNormal")
df_article1_messages.head()

,_id,content,sender,recipients.0,threadId,timestamp,EchantillonNormal
0,FjuMJ37PjJ7DxqnmT,je te passe mon insta et je te donnerai mon nu...,uoHC9sGdWEkgdazZQ,qHRKFnAqv5ipS9mhR,25woAfTMQDGwuNsoS,1.605613e+12,True
1,Fo7kokj7nDMKD2JWr,pas de soucis ;) je peux te passer mon Whats'a...,uoHC9sGdWEkgdazZQ,qHRKFnAqv5ipS9mhR,25woAfTMQDGwuNsoS,1.605613e+12,True
2,B4nkfsKPjNQSR2AZ8,Ah bah je viens de voir que c’est mal vu de de...,qHRKFnAqv5ipS9mhR,uoHC9sGdWEkgdazZQ,25woAfTMQDGwuNsoS,1.605024e+12,True
3,irioQAHermketHEb5,Salut 👋🏼 ton parcours m’inspire énormément et ...,qHRKFnAqv5ipS9mhR,uoHC9sGdWEkgdazZQ,25woAfTMQDGwuNsoS,1.604848e+12,True
4,ys2Q4j7irtk4S2mYh,Ouii\nEt du coup toi pour la suite tu vas fair...,ku5FNNe97ZiNN3mRv,ptBQ9emHw8he5pMNn,2cDBPgc59itjrnGv4,1.600979e+12,True


In [5]:
df_article1_messages=df_article1_messages.drop(["_id","sender","recipients.0","threadId","timestamp","EchantillonNormal"], axis=1)
df_article1_messages["Harmful"]=0

In [6]:
df_article1_messages.head()

,content,Harmful
0,je te passe mon insta et je te donnerai mon nu...,0
1,pas de soucis ;) je peux te passer mon Whats'a...,0
2,Ah bah je viens de voir que c’est mal vu de de...,0
3,Salut 👋🏼 ton parcours m’inspire énormément et ...,0
4,Ouii\nEt du coup toi pour la suite tu vas fair...,0


In [7]:
df_to_moderate["content"]=df_to_moderate["tweet (without @)"]
df_to_moderate=df_to_moderate.drop(["tweet (without @)"], axis=1)
df_to_moderate=df_to_moderate.dropna()
df_to_moderate.tail()

,Harmful,content
195,1.0,nique tes morts sale juif
196,1.0,façon gauchiste c'est comme le choléra ça se r...
197,1.0,j’ai rien contre lui j’rigolais mongol va
198,1.0,les rebeus la peau lisse c ca faiblesse
199,1.0,ce qui tue c'est que meme offset il en marre d...


In [8]:

print(f"Nombre de messages à modérer {df_to_moderate.shape[0]}")

Nombre de messages à modérer 200


In [9]:
df_to_moderate=pd.concat([df_article1_messages[:200],df_to_moderate],ignore_index=True)

In [10]:
df_to_moderate.head()

,content,Harmful
0,je te passe mon insta et je te donnerai mon nu...,0.0
1,pas de soucis ;) je peux te passer mon Whats'a...,0.0
2,Ah bah je viens de voir que c’est mal vu de de...,0.0
3,Salut 👋🏼 ton parcours m’inspire énormément et ...,0.0
4,Ouii\nEt du coup toi pour la suite tu vas fair...,0.0


In [11]:
df_to_moderate.shape

(400, 2)

## Creating the test Dataset

Problem : we do not have a balanced Dataset of real examples from article 1 for which we know the real distribution.

Two proposals : 
Test on ToModerate given by A1, doing a classification by hand --> Too time consuming.

Test on the dataset MLMA. 
See % that come back on Article 1 dataset. 

In [12]:
test=pd.read_csv("fr_dataset_test.csv")
#Dropping Unused columns
cols=[7,8,9]
test=test.drop(test.columns[cols],axis=1)
test=test.drop(columns=["HITId","directness","annotator_sentiment","target","group"])
#Changing sentiment to 0 (for normal) and 1 (for else) 
test["sentiment"]=[0 if test["sentiment"][x]=="normal" else 1 for x in range(test.shape[0])]


test.head()

,tweet,sentiment
0,rt @user personnage ainsi q tte la clique gauc...,1
1,@user @user @user bah oui t'as raison l'autre ...,1
2,moi j'ai personne à qui m'adresser pour réclam...,1
3,@user @user c...est vrai que les anchois sont ...,0
4,eh les renois faut se réveiller la @url,0


## Let's go !

In [13]:
test_values=np.array(test["tweet"])
test_labels=np.array(test["sentiment"])

train_values=np.array(df_to_moderate["content"])
train_values=[str(train_values[i]) for i in range(len(train_values))]
train_labels=np.array(df_to_moderate["Harmful"])

# print([train_values[i] if type(train_values[i])==type(1) else 0 for i in range(len(train_values))])

In [14]:
#Defining a function to do 10 times the log reg

from sklearn.linear_model import SGDClassifier

def evaluate_log_reg(train_features, test_features, y_train, y_test):
    score=0
    for i in range(10):
        log_reg=SGDClassifier(loss="log")
        log_reg.fit(train_features,y_train)
        score+=log_reg.score(test_features,y_test)
    score=score/10

    return score


In [15]:
# Bag of Word

from sklearn.feature_extraction.text import CountVectorizer

bow=CountVectorizer()
train_bow=bow.fit_transform(train_values)
test_bow=bow.transform(test_values)

score=evaluate_log_reg(train_bow, test_bow, train_labels, test_labels)
print(score)


0.7212159434914227


In [16]:
# TDIDF

from sklearn.feature_extraction.text import TfidfVectorizer

tdidf=TfidfVectorizer()
train_tdidf=tdidf.fit_transform(train_values)
test_tdidf=tdidf.transform(test_values)

score=evaluate_log_reg(train_bow, test_bow, train_labels, test_labels)
print(score)

0.723864783047427


### first attempt with Camembert

In [17]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification

tokenizer=CamembertTokenizer.from_pretrained("camembert-base")
# cam_mod=CamembertModel.from_pretrained("camembert-base")
model=CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)

num_labels=2

# train_cam=cam_tok.encode(train_values)
# test_cam=cam_tok.encode(list(test_values), truncation=True)

def preprocess_function(examples):
    return tokenizer(examples, truncation=True)


Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

In [18]:
train_tokenized=tokenizer(train_values)
test_tokenized=tokenizer(test_values.tolist())



In [19]:
# train_df=pd.concat([train_tokenized,train_labels])

In [20]:
from transformers import TrainingArguments, Trainer
batch_size=16
metric_name="accuracy"
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [21]:
from datasets import load_metric

metric_accu = load_metric("accuracy")
print(metric_accu)

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    normalize: If False, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight: Sample weights.
Returns:
    accuracy: Accuracy score.
""", stored examples: 0)


In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric_accu.compute(predictions=predictions, references=labels)

In [23]:
def model_init():
    return CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=num_labels)

In [24]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

In [25]:
# trainer.train()

### Second attempt with camembert

In [26]:
# Remember : 
# test_values=np.array(test["tweet"])
# test_labels=np.array(test["sentiment"])

# train_values=np.array(df_to_moderate["content"])
# train_values=[str(train_values[i]) for i in range(len(train_values))]
# train_labels=np.array(df_to_moderate["Harmful"])



In [27]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_values, train_labels, test_size=.2)

In [28]:
from transformers import CamembertTokenizer
tokenizer=CamembertTokenizer.from_pretrained("camembert-base")

In [29]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_values.tolist(), truncation=True, padding=True)

In [30]:
import torch

class A1Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset =  A1Dataset(train_encodings, train_labels)
val_dataset =  A1Dataset(val_encodings, val_labels)
test_dataset =  A1Dataset(test_encodings, test_labels)

In [31]:
# from transformers import CamembertForSequenceClassification, Trainer, TrainingArguments

# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size per device during training
#     per_device_eval_batch_size=64,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
# )

# model = CamembertForSequenceClassification.from_pretrained("camembert-base")

# trainer = Trainer(
#     model=model,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     eval_dataset=val_dataset             # evaluation dataset
# )

# trainer.train()

In [32]:
from torch.utils.data import DataLoader
from transformers import CamembertForSequenceClassification, AdamW

device =  torch.device('cpu')

model = CamembertForSequenceClassification.from_pretrained('camembert-base')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

In [1]:
model("Essayons celle la")

NameError: name 'model' is not defined